In [ ]:
#default_exp utils

# Utilities
> Various utility functionalities

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#export
import enum
from typing import Any
from functools import wraps, partial

In [ ]:
#export
class _DocumentedEnum(enum.Enum):
    """
    An `Enum` capabile of having its members have docstrings, compatible with the functional API
    """
    def __init__(self, *args):
        """
        Creates a generic enumeration with potential assigning of a member docstring

        Should be passed in the form of:
          value, docstring
        """
        if len(args) == 2:
            # Have value, docstring
            self._value_, self.__doc__ = args
        else:
            # Just value
            self._value_ = args[0]
    
    def __str__(self): return str(self.value)
    def __eq__(self, other): return getattr(other, "value", other) == self.value
    def __ne__(self, other): return getattr(other, "value", other) != self.value

In [ ]:
#hide
_da = [["addition", ("addition", "Sum of two numbers")], ["subtraction", ("subtraction")]]

_d = _DocumentedEnum("test_enum", _da)
test_eq(hasattr(_d, "addition"), True)
test_eq(str(_d.addition), "addition")
test_eq(_d.addition.__doc__, "Sum of two numbers")
test_eq(_d.addition == "addition", True)

test_eq(str(_d.subtraction), "subtraction")
test_eq(_d.subtraction.__doc__, "An enumeration.")
test_eq(_d.subtraction != "addition", True)

In [ ]:
#export
def _assign_annotations(cls, special=None):
    """
    Creates a `_DocumentedEnum` based on annotations in `cls`
    """
    cls_annotations = cls.__dict__.get("__annotations__", {})
    names = []
    for name, typ in list(cls_annotations.items()):
        member = [name]
        value = name.lower()
        cls_annotations.pop(name)
        doc = getattr(cls, name, None)
        if doc is not None:
            member.append((value,doc))
        else:
            member.append(value)
        names.append(member)
    if special is not None:
        for name in special:
            value, doc = getattr(cls, name, None)
            names.append([name, (value, doc)])
    new_cls = _DocumentedEnum(value=cls.__name__, names=names)
    new_cls.__doc__ = cls.__doc__
    return new_cls

In [ ]:
#export
def enumify(cls=None, *, special:list=None):
    """
    A decorator to turn `cls` into an Enum class with member values as property names.
    
    If any Enum member is defined as a regular tuple of (value, doc), they should be passed in
    to `special`
    
    Classes should use the following annotation when defining them:
    ```python
    from typing import Any
    @enumify(special="name_three")
    class MyClass:
      NAME:Any = "Some documented enum value"
      name_two:Any # An undocumented enum value
      name_three = "A value", "Some documentation"
    ```
    """
    def wrap(cls, special=None): return _assign_annotations(cls, special)

    if cls is None and special is None:
        return wrap
    if not isinstance(special, list) and special is not None: 
        special = [special]
    if cls is None:
        return partial(enumify, special=special)
    return wrap(cls, special)

In [ ]:
@enumify(special=["WEDNESDAY"])
class DaysOfWeek:
    MONDAY:Any = "First day of the week"
    TUESDAY:Any
    WEDNESDAY = "Wed", "Second day of the week"
    
test_eq(DaysOfWeek.MONDAY, "monday")
test_eq(DaysOfWeek.MONDAY.__doc__, "First day of the week")
test_eq(DaysOfWeek.TUESDAY, "tuesday")
test_eq(DaysOfWeek.TUESDAY.__doc__, None)
test_eq(DaysOfWeek.WEDNESDAY, "Wed")
test_eq(DaysOfWeek.WEDNESDAY.__doc__, "Second day of the week")